In [1]:
!pip install pyshp

  Running setup.py bdist_wheel for pyshp: started
  Running setup.py bdist_wheel for pyshp: finished with status 'done'
  Stored in directory: C:\Users\Java\AppData\Local\pip\Cache\wheels\b8\a2\56\da058e1e1639061be3399baa32fb82362e2ae1afd8fe0179b0
Successfully built pyshp


https://gist.github.com/frankrowe/6071443

In [9]:
import shapefile
# read the shapefile
reader = shapefile.Reader("COUNTY_MOI_1060525.shp")
fields = reader.fields[1:]
field_names = [field[0] for field in fields]
buffer = []
for sr in reader.shapeRecords():
   atr = dict(zip(field_names, sr.record))
   geom = sr.shape.__geo_interface__
   buffer.append(dict(type="Feature", \
    geometry=geom, properties=atr)) 

# write the GeoJSON file
from json import dumps
geojson = open("pyshp-demo.json", "w")
geojson.write(dumps({"type": "FeatureCollection",\
"features": buffer}, indent=2) + "\n")
geojson.close()


TypeError: Object of type 'bytes' is not JSON serializable

https://gist.github.com/agalea91/c0e0d1897d1d98a0029ac0baa02b9fca

In [8]:
# -*- coding: utf-8 -*-
from json import dumps
import shapefile
import click

@click.command()
@click.option(
    '--f',
    help="Filename",
    default='COUNTY_MOI_1060525.shp'
)
def main(f):
    # read the shapefile
    reader = shapefile.Reader(f)
    fields = reader.fields[1:]
    field_names = [field[0] for field in fields]
    buffer = []
    for sr in reader.shapeRecords():
        record = sr.record
        # Make sure everything is utf-8 compatable
        record = [r.decode('utf-8', 'ignore') if isinstance(r, bytes) 
                  else r for r in record]
        atr = dict(zip(field_names, record))
        geom = sr.shape.__geo_interface__
        buffer.append(dict(type="Feature", geometry=geom, properties=atr)) 

    # write the GeoJSON file
    with open('_'.join(f.split('.')[:-1]) + ".geo.json", "w") as geojson:
        geojson.write(dumps({"type": "FeatureCollection",\
                             "features": buffer}, indent=2) + "\n")

if __name__ == '__main__':
    main()

UnsupportedOperation: not writable